In [ ]:
import sys
sys.path.append("../dependencies/geometric-algebra-transformer")

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from ezgatr.interfaces import point, rotation
from gatr import GATr, SelfAttentionConfig, MLPConfig
from tqdm import tqdm
from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.transforms import random_rotation, matrix_to_quaternion

In [ ]:
def plot_pointcloud(points, title=""):
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(x, y, z)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [ ]:
# Load the dolphin mesh.
device = "cuda:0"
trg_obj = "../data/tmp/dolphin.obj"

# We read the target 3D model using load_obj
verts, faces, aux = load_obj(trg_obj)

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We scale normalize and center the target mesh to fit in a sphere of radius 1 centered at (0,0,0). 
# (scale, center) will be used to bring the predicted mesh to its original center and scale
# Note that normalizing the target mesh, speeds up the optimization but is not necessary!
center = verts.mean(0)
verts = verts - center
scale = max(verts.abs().max(0)[0])
verts = verts / scale

# We construct a Meshes structure for the target mesh
trg_mesh = Meshes(verts=[verts], faces=[faces_idx])

# We initialize the source shape to be a sphere of radius 1
src_mesh = ico_sphere(4, device)

In [ ]:
# model = MVOnlyGATrModel(
#     MVOnlyGATrConfig(
#         num_layers=2,
#         size_channels_in=2,
#         size_channels_hidden=16,
#         norm_channelwise_rescale=False,
#         attn_is_causal=False,
#     )
# )
model = GATr(
    in_mv_channels=2,
    out_mv_channels=1,
    hidden_mv_channels=16,
    in_s_channels=None,
    out_s_channels=None,
    hidden_s_channels=1,
    num_blocks=2,
    attention=SelfAttentionConfig(),  # Use default parameters for attention
    mlp=MLPConfig(),  # Use default parameters for MLP
)
model = model.to(device)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-5,
)

In [ ]:
%matplotlib inline

criterion = nn.MSELoss()
losses = []

loop = tqdm(range(10001))
for i in loop:
    r = random_rotation(dtype=torch.float, device=device)
    x = sample_points_from_meshes(trg_mesh, 1024)
    y = torch.einsum("bni, ji -> bnj", x.detach(), r)

    x = point.encode(x).unsqueeze(-2)
    r = (
        rotation.encode(matrix_to_quaternion(r)[..., [1, 2, 3, 0]])
        .reshape(1, 1, 1, 16)
        .repeat((1, 1024, 1, 1))
    )
    p, _ = model(torch.cat([x, r], dim=-2), scalars=None)
    # p = model(torch.cat([x, r], dim=-2))
    p = point.decode(p).squeeze(-2)
    loss = criterion(p, y)
    losses.append(float(loss.detach().cpu()))

    loop.set_description('total_loss = %.6f' % loss)
    if i % 2000 == 0:
        plot_pointcloud(p, title="iter: %d" % i)
        plot_pointcloud(y, title="iter: %d" % i)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()